# AI Agent 개발하기 (1) 

## AI Agent의 구성요소 

AI Agent란 특정 목표를 달성하기 위해 환경과 상호작용하며 자율적으로 행동하는 인공지능 시스템입니다.

LLM(Large Language Model, 예: ChatGPT)은  주어진 텍스트 입력에 대해 자연어 처리를 통해 응답을 생성하는 모델로, 주로 대화나 텍스트 생성에 특화되어 있으며, 환경과의 지속적인 상호작용보다는 주어진 입력에 대한 응답 생성에 중점을 둡니다. 하지만 AI Agent는 환경과 지속적으로 상호작용하며, 지각, 두뇌, 행동 요소를 가집니다.

AI Agent는 3가지 요소로 구성되어 있습니다.

<img src="../resources/assets/agents-components.png" width="40%">

1. Perception (지각): AI 에이전트가 외부 환경으로부터 정보를 수집하고 인식하는 과정.
2. Brain (두뇌): 수집된 정보를 처리하고 의사결정을 내리는 AI 에이전트의 중앙 처리 시스템.
3. Action (행동): 두뇌에서 내린 의사결정에 따라 AI 에이전트가 환경에 반응하고 행동을 수행하는 과정.



### SETUP

#### OpenAI API KEY를 환경변수에 등록하기

```text
OPENAI_API_KEY=sk-prod-xxx
````

In [1]:
import dotenv
import os

dotenv.load_dotenv("../.env")

if not "OPENAI_API_KEY" in os.environ:
    raise Exception("OPENAI_API_KEY가 환경변수에 존재하지 않습니다.")

#### Langchain OpenAI 불러오기

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## 간단한 AI Agent 봇 만들기 : ShellBot

> 유저의 요청에 따라 Shell 명령어를 작성 후, 실행하는 AI Agent 봇 만들기

위의 목적을 수행하기 위해서는 크게 3단계가 필요합니다.

1. Shell 명령어 생성 요청하는 프롬프트 만들기
2. 프롬프트에 대한 응답값을 파싱하고, 이를 실행하기
3. 실행한 결과를 AI Agent에 전달 후 결과를 받기

### 1.Shell 명령어를 생성하도록 요청하는 프롬프트 만들기

LLM 모델들은 Text 기반으로 동작합니다. 입력값도 Text로 받고, 출력값도 Text로 출력합니다.

우리가 원하는 형태의 출력값을 구성하기 위해서는 우리는 입력 프롬프트를 그에 맞게 구성해야 합니다.

**목표**
> 유저의 요청에 따라, 실행가능한 Shell 명령어를 작성하기

**예시코드**

1. `jupyter notebook을 실행하고 있는 pid를 찾아주세요`
2. `jupyter notebook의 ip와 port를 가져와 주세요`
3. `현재 컴퓨터에 할당된 외부 IP와 wifi 내 할당된 내부 IP를 각각 보여 주세요`

In [3]:
import platform
current_platform = platform.platform()
language = 'shell'

TEST_PROMPT1 = 'jupyter notebook을 실행하고 있는 pid를 찾아주세요'
TEST_PROMPT2 = '현재 동작중인 jupyter notebook의 ip와 port를 가져와 주세요'
TEST_PROMPT3 = '현재 컴퓨터에 할당된 외부 IP와 wifi 내 할당된 내부 IP를 각각 보여 주세요'

우리가 받고 싶은 응답값은 Shell Script Code 입니다. 이를 위해서는 프롬프트 입력에 필수적으로 "어떻게 출력값을 반환할지"를 요청해야 합니다. 

In [8]:
from typing import Dict, Any
from langchain.schema.output_parser import BaseOutputParser

class CodeOutputParser(BaseOutputParser[Dict[str, Any]]):
    """코드 블록과 언어 정보를 추출하는 커스텀 OutputParser"""

    def parse(self, text: str) -> Dict[str, Any]:
        import re
        # 마크다운 코드 블록 패턴
        pattern = r'```(\w+)?\s*([\s\S]+?)\s*```'
        matches = re.findall(pattern, text, re.MULTILINE)

        if not matches:
            raise ValueError("No code block found in the output")

        # 첫 번째 코드 블록 사용
        language, code = matches[0]

        # 언어가 지정되지 않은 경우 'unknown'으로 설정
        language = language.strip() if language else 'unknown'
        code = code.strip()

        return {
            "language": language,
            "code": code
        }

    def get_format_instructions(self) -> str:
        return (
            "Your response should be formatted as a Markdown code block. "
            "Specify the programming language after the opening triple backticks. "
            "For example:\n\n"
            "```python\n"
            "print('Hello, World!')\n"
            "```"
        )

In [9]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

code_output_parser = CodeOutputParser()
format_instructions = code_output_parser.get_format_instructions()
print(format_instructions)

Your response should be formatted as a Markdown code block. Specify the programming language after the opening triple backticks. For example:

```python
print('Hello, World!')
```


#### System Message

시스템 메시지에서는 
1. 시스템의 역할 (role)
2. 지켜야할 규칙
3. 출력값의 형태

를 지정합니다.

In [10]:
from langchain_core.prompts import SystemMessagePromptTemplate

system_template = """You are a helpful code assistant. Generate executable code that fulfills the user's goal.
- Include a code block that addresses the user’s need.
- Provide a single, error-free code block
- Display output (stdout) for result verification if applicable.
- Programming language: {language}.
- Current platform: {current_platform}.

{format_instructions}"""

system_message = SystemMessagePromptTemplate.from_template(system_template)

In [11]:
print("프롬프트 >>")

print(system_message.format(
    language=language,
    current_platform=current_platform,
    format_instructions=format_instructions
).content)

프롬프트 >>
You are a helpful code assistant. Generate executable code that fulfills the user's goal.
- Include a code block that addresses the user’s need.
- Provide a single, error-free code block
- Display output (stdout) for result verification if applicable.
- Programming language: shell.
- Current platform: macOS-14.6.1-arm64-arm-64bit.

Your response should be formatted as a Markdown code block. Specify the programming language after the opening triple backticks. For example:

```python
print('Hello, World!')
```


#### 2. Human Message

휴먼메시지에서는 우리가 실행하길 원하는 목표를 지정합니다. 

In [14]:
from langchain_core.prompts import HumanMessagePromptTemplate

human_template = '''goal: {goal}'''
human_message = HumanMessagePromptTemplate.from_template(human_template)

#### 3. 시스템 메시지와 휴먼 메시지를 채팅 프롬프트로 묶기

System Message와 Human Message를 채팅과 같은 구조로 묶어줍니다. 


In [15]:
from langchain_core.prompts import ChatPromptTemplate

code_generation_prompt = ChatPromptTemplate(
    messages=[system_message, human_message],
    input_variables=["goal", "language"],
    partial_variables={"format_instructions": format_instructions, "current_platform": current_platform}
)

In [17]:
prompt_value = code_generation_prompt.format_prompt(
    goal='현재 시간을 출력해주세요',
    language='node'
)

messages = prompt_value.to_messages()

print("생성된 프롬프트>>>")
for message in messages:
    print(f"[{message.type}]\n{message.content}\n")

생성된 프롬프트>>>
[system]
You are a helpful code assistant. Generate executable code that fulfills the user's goal.
- Include a code block that addresses the user’s need.
- Provide a single, error-free code block
- Display output (stdout) for result verification if applicable.
- Programming language: node.
- Current platform: macOS-14.6.1-arm64-arm-64bit.

Your response should be formatted as a Markdown code block. Specify the programming language after the opening triple backticks. For example:

```python
print('Hello, World!')
```

[human]
goal: 현재 시간을 출력해주세요



### 2. 프롬프트를 LLM 모델에 넣고 실행하기

LLM 모델에 담아서 호출합니다 

In [18]:
model_output = model.invoke(messages)
model_output

AIMessage(content='```javascript\nconst now = new Date();\nconsole.log(now);\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 132, 'total_tokens': 146, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bfc2c88a-671e-47f7-aaa7-5a7b4d47bc25-0', usage_metadata={'input_tokens': 132, 'output_tokens': 14, 'total_tokens': 146})

### 3. LLM 출력값을 파싱하기

위의 응답값을 파싱하면 아래와 같이 출력됩니다.

In [19]:
parsed_output = code_output_parser.parse(model_output.content)
parsed_output

{'language': 'javascript',
 'code': 'const now = new Date();\nconsole.log(now);'}

In [21]:
code_output_parser.invoke(model_output)

{'language': 'javascript',
 'code': 'const now = new Date();\nconsole.log(now);'}

### 4. 위의 코드를 실제로 실행하기

위에서 파싱한 결과를 넣어서 처리하면 아래와 같습니다 

In [20]:
from langchain.schema.runnable import RunnableSerializable
from typing import Dict, Any, Optional
import tempfile
import logging
import subprocess
import tempfile
logging.basicConfig()


class CodeExecutor(RunnableSerializable):
    def invoke(self, inputs: Dict[str, Any], config:Optional[Dict[str,Any]]=None) -> Dict[str, Any]:
        code = inputs['code']
        language = inputs['language']
        
        result = self._execute(code, language)
        return {
            "code": code,
            "execution_result": result
        }

    def _execute(self, code_block:str, language: str):
        """ 주어진 Code block을 실행시키기 """
        def _write_and_run(execute_command, code):
            with tempfile.NamedTemporaryFile(mode='w') as temp_file:
                temp_file.write(code)
                temp_file.flush()            
                temp_file_path = temp_file.name
                return subprocess.run(f'{execute_command} "{temp_file_path}"', shell=True, capture_output=True, text=True)
        
        if language in ('shell', 'bash'):
            output = _write_and_run('bash', code_block)
        elif language in ('python', 'python3'):
            output = _write_and_run('python', code_block)
        elif language in ('node', 'javascript'):
            output = _write_and_run('node', code_block)
        else:
            raise ValueError(f"Not Available language: {language}")
            
        return {
            "returncode": output.returncode,
            "result": output.stderr.strip() if output.returncode else output.stdout.strip(),
        }

In [22]:
code_executor = CodeExecutor()

code_executor.invoke(parsed_output)

{'code': 'const now = new Date();\nconsole.log(now);',
 'execution_result': {'returncode': 0,
  'result': '\x1b2024-09-24T13:09:15.006Z\x1b'}}

## 다른 명령어들 실행해보기

Langchain에서는 여러 컴포넌트를 Chain으로 정의 후 구성할 수 있습니다 

In [23]:
chain = (
    code_generation_prompt # LLM 모델에 넣을 프롬프트 생성
    | model # LLM 모델 호출
    | code_output_parser # LLM 모델의 출력값 파싱
    | code_executor # 코드를 실행시키기
)

In [24]:
chain.invoke({
    'goal': "외부 IP 주소를 알려줘",
    'language': "shell"
})

{'code': 'curl ifconfig.me',
 'execution_result': {'returncode': 0, 'result': '218.48.227.56'}}

In [25]:
chain.invoke({
    'goal': "외부 IP 주소를 알려줘",
    'language': "python"
})

{'code': 'import socket\n\n# Get the external IP address using a socket connection\nexternal_ip = socket.gethostbyname(socket.gethostname())\n\nprint("External IP Address:", external_ip)',
 'execution_result': {'returncode': 0,
  'result': 'External IP Address: 127.0.0.1'}}

In [26]:
chain.invoke({
    'goal': "현재 컴퓨터의 외부 IP 주소를 알려줘",
    'language': "node"
})

{'code': "const { exec } = require('child_process');\n\nexec('curl ifconfig.me', (error, stdout, stderr) => {\n    if (error) {\n        console.error(`exec error: ${error}`);\n        return;\n    }\n    console.log(`External IP Address: ${stdout}`);\n});",
 'execution_result': {'returncode': 0,
  'result': 'External IP Address: 218.48.227.56'}}

In [27]:
chain.invoke({
    'goal': "/Users/blockgrammer/Downloads 경로 하위의 pdf 파일 목록을 가져와줘",
    'language': "python"
})

{'code': 'import os\n\n# Directory path\npath = "/Users/blockgrammer/Downloads"\n\n# List all PDF files in the directory\npdf_files = [f for f in os.listdir(path) if f.endswith(\'.pdf\')]\n\n# Print the list of PDF files\nfor pdf_file in pdf_files:\n    print(pdf_file)',
 'execution_result': {'returncode': 0,
  'result': 'TTT (1).pdf\nTTT.pdf\nPMS 설계.pdf\nTTT (2).pdf\nTTT (3).pdf'}}

In [29]:
chain.invoke({
    'goal': "크롬 켜서 구글에 강상재 검색해줘",
    'language': "python"
})

{'code': "import webbrowser\n\n# Open Chrome and search for 강상재 on Google\nwebbrowser.get('chrome').open_new_tab('https://www.google.com/search?q=강상재')",
 'execution_result': {'returncode': 0, 'result': ''}}

In [38]:
chain.invoke({
    'goal': "구글 캘린더에 내일 오후 4시로 최선열이랑 미팅 일정 추가해줘",
    'language': "python"
})

{'code': 'from datetime import datetime, timedelta\nimport os\n\n# Set the meeting details\nmeeting_title = "Meeting with 최선열"\nmeeting_time = datetime.now().replace(hour=16, minute=0) + timedelta(days=1)\n\n# Format the meeting time for Google Calendar\nmeeting_time_str = meeting_time.strftime("%Y-%m-%dT%H:%M:%S")\nmeeting_end_time_str = (meeting_time + timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S")\n\n# Create the event in Google Calendar\nos.system(f"open \'http://www.google.com/calendar/event?action=TEMPLATE&text={meeting_title}&dates={meeting_time_str}/{meeting_end_time_str}\'")',
 'execution_result': {'returncode': 0, 'result': ''}}

In [50]:
chain.invoke({
    'goal': "넷플릭스에 겨울왕국 틀어줘",
    'language': "python"
})

{'code': 'import webbrowser\n\n# Search for "Frozen" on Netflix\nquery = "Frozen"\nurl = f"https://www.netflix.com/search?q={query}"\nwebbrowser.open(url)',
 'execution_result': {'returncode': 0, 'result': ''}}